# **🎮 Steam 게임 데이터 수집 및 분석 프로젝트**
## **📌 프로젝트 목표**
Steam API와 SteamSpy를 이용해 5만 개 이상의 게임 관련 데이터를 수집하고, 이를 기반으로 다음과 같은 분석을 진행한다:

출시일, 가격, 무료 여부, 장르 정보 분석

장르별 출시 트렌드, 가격대별 유저 선호도

향후 판매 예측을 위한 모델링 기초 데이터 확보

## **📊 분석 프로세스 개요**
단계	설명
1. AppID 전체 목록 수집	GetAppList API로 모든 AppID 확보
2. 유효 게임 필터링	type == game 조건으로 비게임 제거
3. 상세 정보 수집	appdetails로 출시일, 가격, 장르 등 확보
4. 1만 개 단위로 저장	진행 중간 CSV 저장으로 백업 및 추적 가능
5. 이후 SteamSpy 결합	판매량, 유저 수 등 외부 정보 보강 예정

## **🧾 주요 필터 기준**
type == "game" 인 경우만 수집

에러 또는 응답 실패 시 패스

한국 시간 기준 0.2초 간격 요청

## **🧠 분석 코드**

In [2]:
import requests
import pandas as pd
import time
import os

In [3]:
def get_all_steam_apps():
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    app_list = response.json()['applist']['apps']
    df_apps = pd.DataFrame(app_list)
    return df_apps

# 전체 앱 목록 수집
df_apps = get_all_steam_apps()

# CSV 저장
df_apps.to_csv("steam_applist.csv", index=False)

print(f"✅ 앱 목록 수집 완료: {len(df_apps)}개 AppID 저장됨")
df_apps.head()

✅ 앱 목록 수집 완료: 247588개 AppID 저장됨


,appid,name
0,2006441,
1,2005791,
2,216938,Pieterw test app76 ( 216938 )
3,660010,test2
4,660130,test3


In [4]:
df_apps = pd.read_csv("steam_applist.csv")

In [5]:
df_apps

,appid,name
0,2006441,NaN
1,2005791,NaN
2,216938,Pieterw test app76 ( 216938 )
3,660010,test2
4,660130,test3
...,...,...
247583,3194360,Formula Legends
247584,2003640,Village Heroes
247585,2221050,Neanderthallica
247586,1619520,Cross Blitz


In [6]:
def get_app_details(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get(str(appid), {}).get('success'):
                return data[str(appid)]['data']
    except Exception as e:
        print(f"[ERROR] appid {appid} - {e}")
    return None

def collect_game_data_resume(df_apps, target_count=50000, save_block_size=10000, save_interval=100, autosave_path="data_blocks/_autosave_partial.csv"):
    collected = []
    saved_blocks = 0
    total_checked = 0
    last_saved_count = 0

    os.makedirs("data_blocks", exist_ok=True)

    # 타이머 초기화
    start_time = time.time()
    last_checkpoint_time = start_time

    # 이전에 저장된 데이터 불러오기
    if os.path.exists(autosave_path):
        print(f"[RESUME] Autosave 발견됨: {autosave_path} 로드 중...")
        collected_df = pd.read_csv(autosave_path)
        collected = collected_df.to_dict('records')
        collected_appids = set(d['appid'] for d in collected)
        saved_blocks = len(collected) // save_block_size
        last_saved_count = len(collected)
        print(f"[RESUME] {len(collected)}개의 게임 데이터를 불러왔습니다. 이어서 수집합니다.")
    else:
        collected_appids = set()
        print("[START] 새 수집 시작")

    appids = df_apps['appid'].tolist()

    for appid in appids:
        if appid in collected_appids:
            continue

        details = get_app_details(appid)
        total_checked += 1

        if details and details.get('type') == 'game':
            collected.append({
                'appid': appid,
                'name': details.get('name'),
                'release_date': details.get('release_date', {}).get('date'),
                'is_free': details.get('is_free'),
                'type': details.get('type'),
                'genres': details.get('genres'),
                'price_overview': details.get('price_overview'),
            })

            # 블록 단위 저장
            if len(collected) >= (saved_blocks + 1) * save_block_size:
                block_df = pd.DataFrame(collected[saved_blocks * save_block_size : (saved_blocks + 1) * save_block_size])
                filename = f"data_blocks/steam_games_block_{saved_blocks+1}.csv"
                block_df.to_csv(filename, index=False)
                print(f"[SAVE] {filename} 저장됨 - {len(block_df)}개 항목")
                saved_blocks += 1

            # 100개 단위 자동 저장
            if (len(collected) - last_saved_count) >= save_interval:
                df_partial = pd.DataFrame(collected)
                df_partial.to_csv(autosave_path, index=False)

                current_time = time.time()
                block_elapsed = current_time - last_checkpoint_time
                total_elapsed = current_time - start_time
                last_checkpoint_time = current_time
                last_saved_count = len(collected)

                print(f"\n[AUTOSAVE] {len(df_partial)}개 저장됨 (중간 백업)")
                print(f"[INFO] 수집된 게임 수: {len(collected)} / 검사한 App 수: {total_checked}")
                print(f"       이번 {save_interval}개 수집 처리 시간: {block_elapsed // 60:.0f}분 {block_elapsed % 60:.0f}초")
                print(f"       총 누적 시간: {total_elapsed // 60:.0f}분 {total_elapsed % 60:.0f}초\n")

        if len(collected) >= target_count:
            print(f"[TARGET] {target_count}개 도달, 수집 종료")
            break

        time.sleep(0.1)

    # 마지막 블록 저장
    if len(collected) > saved_blocks * save_block_size:
        remaining_df = pd.DataFrame(collected[saved_blocks * save_block_size:])
        filename = f"data_blocks/steam_games_block_{saved_blocks+1}_final.csv"
        remaining_df.to_csv(filename, index=False)
        print(f"[FINAL SAVE] {filename} 저장됨 - {len(remaining_df)}개 항목")

    if len(collected) < target_count:
        print(f"[WARNING] AppID 전체를 검사했지만 {target_count}개를 수집하지 못했습니다.")
        print(f"          최종 수집된 게임 수: {len(collected)}개 / 전체 검사 수: {total_checked}")

    print(f"[DONE] 총 {len(collected)}개 게임 수집 완료 (전체 AppID {total_checked}개 확인)")
    return pd.DataFrame(collected)

In [ ]:
# 전체 앱 리스트 불러오기 (사전 다운로드한 CSV 기준)
df_apps = pd.read_csv("steam_applist.csv")

# 수집 실행 (중단 이어서 가능)
df_result = collect_game_data_resume(df_apps, target_count=50000)

[RESUME] Autosave 발견됨: data_blocks/_autosave_partial.csv 로드 중...
[RESUME] 8261개의 게임 데이터를 불러왔습니다. 이어서 수집합니다.
